<h1> # Essai de webscrapping de trustpilot 2023 10 01 LH</h1>

<h3> Objectif </h3>
collecter les avis et notes des utilisateurs concernant une catégories d'entreprises




3) Scrap 2:
Utiliser la liste des entreprises pour scrapper sur chaque page entreprise:
Avis:
- Nom utilisateur/ice
- Nombre d'avis
- Localisation
- Date de l'avis
- Titre de l'avis
- Contenu de l'avis
- Date de l'expérience

<u>-> Sauvegarder en DF/CSV "avis"</u>

In [1]:
# librairies
import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd
import random
import math

<h1> 1. Utiliser la liste des entreprises pour scrapper sur chaque page entreprise </h1>

In [2]:
liste_entreprises = pd.read_csv('../data/liste_entreprises_banque.csv')

In [9]:
liste_entreprises.sort_values("n_avis")
liste_entreprises = liste_entreprises[liste_entreprises.n_avis != "na"]

In [34]:
# enlever de la liste les pages entreprises déjà scrappées
general_df = pd.read_csv("../data/avis/general_df.csv")
liste_entreprises['nom'] = liste_entreprises['nom'].str.replace(" ","")
deja_scrap = general_df.page.value_counts().keys().tolist()
liste_entreprises = liste_entreprises[~liste_entreprises.nom.isin(deja_scrap)]

liste_entreprises.sort_values("n_avis")

,Unnamed: 0,nom,score,n_avis,url_page_entreprise
52,52,SociétéGénérale&Vous,"1,7",1 776,/review/www.societegenerale.fr
19,19,LouvreBanquePrivée,"4,7",215,/review/bpe.fr
33,33,CourtiersPrivés,"3,8",27,/review/www.courtiers-prives.com
0,0,Immoprêt,"4,9",5 305,/review/immopret.fr


In [33]:
# essai de boucle pour prendre les avis sur plusieurs page (en cours)
"""
essayer de scrapper sur la premiere page
- infos de renforcement de la liste entreprises
- avis sur l'liste_entreprise_temp

puis:

faire des boucles en cascade
niveau 1: page entreprise
scrapper le renforcement
mettre counter à 0
estimer le nombre de pages max
    niveau 2, pages de l'entreprise en fonction du nombre de pages max: range (1, pages_max)

"""

"""
-> Resultat: Tous les avis disponibles ont été scrappés pour Banques = 170 000 avis
"""
liste_avis = pd.DataFrame()


for i, page_key in enumerate(liste_entreprises.url_page_entreprise[2:]):
  # nombre de pages extrait depuis page 1
    scrap_url =  "https://fr.trustpilot.com"+ page_key
    print("landing page url", scrap_url)
    landing_html = requests.get(scrap_url)
    landing_soup = BeautifulSoup(landing_html.content, "lxml")
    if landing_soup is None:
        ("landing page is None, we pass")
        pass
    n_avis_page = landing_soup.find("p", class_ = "typography_body-l__KUYFJ typography_appearance-default__AAY17").text
    n_avis_page = n_avis_page.replace('Total : ','').replace('\u202f', '').replace(',', '').strip()
    n_avis_page = int(n_avis_page)
    n_pages = math.ceil(n_avis_page / 20)

    nom_page = landing_soup.find("span", class_ = "typography_display-s__qOjh6 typography_appearance-default__AAY17 title_displayName__TtDDM")
    nom_page = nom_page.text
    nom_page = (nom_page.encode('ascii', 'ignore')).decode("utf-8")
    nom_page = nom_page.replace(" ","")
    nom_page = nom_page.replace("/","_")
    print("\n",nom_page)    #n_pages = 156 # hardcodage pour test
    if len(nom_page) < 2:
        nom_page = "filler"+(random.randrange(20))

    # boucle pour scrapper les différentes pages de l'entreprise
    for i, page in enumerate(range (0,n_pages)):
        scrap_url_n_page = scrap_url+"?page="+str(i+1)
        # aquisition de la page
        print(scrap_url_n_page)

        html = requests.get(scrap_url_n_page)
        page_n = BeautifulSoup(html.content, "lxml")

        nom, user, etoiles, n_avis, avis, localisation =[],[],[],[],[], []
        date_avis, titre_avis,text_avis, date_experience, text_avis = [],[],[],[],[]

        cartes = page_n.find_all("div", class_ = "styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ")
        text_avis = []
        for items in cartes:
            # si la carte d'avis est flaguée passe à la prochaine itération (rare)
            warning = items.find("p", class_="typography_heading-xxs__QKBS8 typography_appearance-default__AAY17 banner-alert_expandable-title__RpT5M")
            if warning is not None: 
                continue            
            # text avis
            text_avis_html =[]
            text_avis_html = items.find("p", attrs={"class": "typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn",
                                            "data-service-review-text-typography" : "true"} )
            if text_avis_html is not None:
                text_avis.append(text_avis_html.text)
            else:
                text_avis.append('na')

            # Nom utilisateur/ice
            utilisateur_html = []
            utilisateur_html = items.find("span", attrs= {"class" : 'typography_heading-xxs__QKBS8 typography_appearance-default__AAY17',
                                                "data-consumer-name-typography":"true"})
            if utilisateur_html is not None:
                user.append(utilisateur_html.text)
            else:
                user.append("na")

            # etoiles
            etoiles_html = []
            etoiles_html = items.find("div", class_ = "styles_reviewHeader__iU9Px")
            if etoiles_html is not None:
                etoiles.append(etoiles_html['data-service-review-rating'])
            else:
                etoiles.append('na')

            # Nombre d'avis
            n_avis_html =[]
            n_avis_html = items.find("div", class_ = "styles_consumerExtraDetails__fxS4S")
            if n_avis_html is not None:
                n_avis.append(n_avis_html["data-consumer-reviews-count"])
            else: 
                n_avis.append('na')

            # Localisation
            localisation_html =[]
            localisation_html = items.find("div", class_= "typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua")
            if localisation_html is not None:
                localisation.append(localisation_html.text)
            else:
                localisation.append('na')

            # Date de l'avis
            date_avis_html = []
            date_avis_html = items.find("div", class_="typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH")
            if date_avis_html is not None:
                date_avis.append(date_avis_html.time['datetime'])
            else:
                date_avis.append('na')

            # Titre de l'avis
            titre_avis_html = []
            titre_avis_html = items.find('h2', class_="typography_heading-s__f7029 typography_appearance-default__AAY17")
            if titre_avis_html is not None:
                titre_avis.append(titre_avis_html.text)
            else:
                titre_avis.append('na')

            # Date de l'expérience
            date_experience_html = []
            date_experience_html = items.find("p", attrs={"class": "typography_body-m__xgxZ_ typography_appearance-default__AAY17",
                                                                "data-service-review-date-of-experience-typography":"true"})
            if date_experience_html is not None:
                date_experience.append(date_experience_html.text)
            else:
                date_avis.append('na')

        # consolidation en dataframe
        liste_avis_temp = pd.DataFrame({"user": user,
                                "etoiles": etoiles,
                                "n_avis": n_avis,
                                "localisation": localisation,
                                "date_avis": date_avis, 
                                "titre_avis": titre_avis, 
                                "text_avis": text_avis,
                                "date_experience": date_experience
        })
        liste_avis_temp['page'] = nom_page
        liste_avis = pd.concat([liste_avis, liste_avis_temp])

        # faire une pause entre chaque page d'entreprise pour éviter de se faire ban
        print("page",page+1,"passée")
        # reset de la page 
        import time
        time.sleep(2)
    liste_avis.to_csv("../data/avis/csv_entreprises/"+nom_page+".csv")
    liste_avis=pd.DataFrame()


landing page url https://fr.trustpilot.com/review/www.courtiers-prives.com

 CourtiersPrivs
https://fr.trustpilot.com/review/www.courtiers-prives.com?page=1
page 1 passée
https://fr.trustpilot.com/review/www.courtiers-prives.com?page=2
page 2 passée
landing page url https://fr.trustpilot.com/review/hellobank.fr

 Hellobank!France
https://fr.trustpilot.com/review/hellobank.fr?page=1
page 1 passée
https://fr.trustpilot.com/review/hellobank.fr?page=2
page 2 passée
https://fr.trustpilot.com/review/hellobank.fr?page=3
page 3 passée
https://fr.trustpilot.com/review/hellobank.fr?page=4
page 4 passée
https://fr.trustpilot.com/review/hellobank.fr?page=5
page 5 passée
https://fr.trustpilot.com/review/hellobank.fr?page=6
page 6 passée
https://fr.trustpilot.com/review/hellobank.fr?page=7
page 7 passée
https://fr.trustpilot.com/review/hellobank.fr?page=8
page 8 passée
https://fr.trustpilot.com/review/hellobank.fr?page=9
page 9 passée
https://fr.trustpilot.com/review/hellobank.fr?page=10
page 10 pas